# Import & Load CASEN household data

### Config

In [1]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

In [2]:
remote_path = 'http://pacha.datawheel.us/datachile/economy/casen/analysis/3_household/'
local_path = '../data/'

In [3]:
import postgres #from local file postgres.py
from commons import inline_table_xml, inline_dimension_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load data

In [4]:
df1 = download_file(remote_path,local_path,'household_for_db.csv')
list(df1)

Downloading... http://pacha.datawheel.us/datachile/economy/casen/analysis/3_household/household_for_db.csv


['comuna_id',
 'zone_id',
 'household_type',
 'walls_material',
 'floor_material',
 'ceiling_material',
 'households_in_land',
 'land_using',
 'household_sq_meters',
 'household_using',
 'family_member_owner',
 'family_member_owner_2',
 'subsidy_or_program',
 'credit',
 'paying_credit',
 'water_source',
 'water_distribution',
 'waste_disposal',
 'electricity',
 'rooms',
 'bathrooms',
 'families_in_household',
 'reason_to_share',
 'cooking_energy_source',
 'heating_energy_source',
 'hot_water_energy_source',
 'less_than_8_blocks_public_transport',
 'less_than_20_blocks_educational_center',
 'less_than_20_blocks_health_center',
 'less_than_20_blocks_market',
 'less_than_20_blocks_atm',
 'less_than_20_blocks_sports_center',
 'less_than_20_blocks_green_areas',
 'less_than_20_blocks_community_equipment',
 'less_than_20_blocks_pharmacy',
 'affected_by_acoustic_contamination',
 'affected_by_air_contamination',
 'affected_by_river_or_lake_contamination',
 'affected_by_public_water_source_conta

In [7]:
df1

,comuna_id,zone_id,household_type,walls_material,floor_material,ceiling_material,households_in_land,land_using,household_sq_meters,household_using,...,affected_by_air_contamination,affected_by_river_or_lake_contamination,affected_by_public_water_source_contamination,affected_by_graffitis_or_advertising,affected_by_accumulation_of_trash_public_areas,affected_by_animal_or_insect_plague,expr,expc,comuna_datachile_id,year
0,1101,1,2,1,4,2,1,5,4,5,...,1,1,1,1,3,3,104,95,113,2015
1,1101,1,2,1,4,2,1,5,4,5,...,1,1,1,1,3,3,104,95,113,2015
2,1101,1,2,1,4,2,1,5,4,5,...,1,1,1,1,3,3,104,95,113,2015
3,1101,1,2,1,4,2,1,5,4,5,...,1,1,1,1,3,3,104,95,113,2015
4,1101,1,2,1,4,2,1,5,4,5,...,1,1,1,1,3,3,104,95,113,2015
5,1101,1,2,4,1,1,1,6,2,6,...,1,1,1,1,4,4,104,95,113,2015
6,1101,1,2,1,1,2,6,5,2,5,...,1,1,1,1,1,1,104,95,113,2015
7,1101,1,3,3,1,1,1,5,3,5,...,2,1,1,2,3,3,104,95,113,2015
8,1101,1,3,3,1,1,1,5,3,5,...,2,1,1,2,3,3,104,95,113,2015
9,1101,1,3,3,1,1,1,6,3,6,...,3,1,1,3,2,3,104,95,113,2015


### Change yes/no/nan responses to match SharedDimension

In [5]:
df1 = df1.astype({'less_than_8_blocks_public_transport':'int','less_than_20_blocks_educational_center':'int','less_than_20_blocks_health_center':'int','less_than_20_blocks_market':'int','less_than_20_blocks_atm':'int','less_than_20_blocks_sports_center':'int','less_than_20_blocks_green_areas':'int','less_than_20_blocks_community_equipment':'int','less_than_20_blocks_pharmacy':'int'})

df1['less_than_8_blocks_public_transport'] = df1['less_than_8_blocks_public_transport'].map({1: 2, 2: 1, 9:0})
df1['less_than_20_blocks_educational_center'] = df1['less_than_20_blocks_educational_center'].map({1: 2, 2: 1, 9:0})
df1['less_than_20_blocks_health_center'] = df1['less_than_20_blocks_health_center'].map({1: 2, 2: 1, 9:0})
df1['less_than_20_blocks_market'] = df1['less_than_20_blocks_market'].map({1: 2, 2: 1, 9:0})
df1['less_than_20_blocks_atm'] = df1['less_than_20_blocks_atm'].map({1: 2, 2: 1, 9:0})
df1['less_than_20_blocks_sports_center'] = df1['less_than_20_blocks_sports_center'].map({1: 2, 2: 1, 9:0})
df1['less_than_20_blocks_green_areas'] = df1['less_than_20_blocks_green_areas'].map({1: 2, 2: 1, 9:0})
df1['less_than_20_blocks_community_equipment'] = df1['less_than_20_blocks_community_equipment'].map({1: 2, 2: 1, 9:0})
df1['less_than_20_blocks_pharmacy'] = df1['less_than_20_blocks_pharmacy'].map({1: 2, 2: 1, 9:0})

### Fill and load

In [6]:
df1[['family_member_owner']] = df1[['family_member_owner']].fillna(value=-1)
df1[['family_member_owner_2']] = df1[['family_member_owner_2']].fillna(value=-1)
df1[['subsidy_or_program']] = df1[['subsidy_or_program']].fillna(value=-1)
df1[['credit']] = df1[['credit']].fillna(value=-1)
df1[['paying_credit']] = df1[['paying_credit']].fillna(value=-1)
df1[['reason_to_share']] = df1[['reason_to_share']].fillna(value=-1)
df1[['zone_id']] = df1[['zone_id']].fillna(value=-1)

df1 = df1.astype({'year':'int','zone_id':'int','comuna_datachile_id':'int','household_type':'int','walls_material':'int','floor_material':'int','ceiling_material':'int','households_in_land':'int','land_using':'int','household_sq_meters':'int','household_using':'int','water_source':'int','water_distribution':'int','waste_disposal':'int','electricity':'int','rooms':'int','bathrooms':'int','families_in_household':'int','cooking_energy_source':'int','heating_energy_source':'int','hot_water_energy_source':'int','less_than_8_blocks_public_transport':'int','less_than_20_blocks_educational_center':'int','less_than_20_blocks_health_center':'int','less_than_20_blocks_market':'int','less_than_20_blocks_atm':'int','less_than_20_blocks_sports_center':'int','less_than_20_blocks_green_areas':'int','less_than_20_blocks_community_equipment':'int','less_than_20_blocks_pharmacy':'int','affected_by_acoustic_contamination':'int','affected_by_air_contamination':'int','affected_by_river_or_lake_contamination':'int','affected_by_public_water_source_contamination':'int','affected_by_graffitis_or_advertising':'int','affected_by_accumulation_of_trash_public_areas':'int','affected_by_animal_or_insect_plague':'int','expr':'int','expc':'int','family_member_owner':'int','family_member_owner_2':'int','subsidy_or_program':'int','credit':'int','paying_credit':'int','reason_to_share':'int'})

engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df1, 'environment', 'fact_household_casen')

DROP TABLE IF EXISTS environment.fact_household_casen;
CREATE TABLE "environment"."fact_household_casen" (
"comuna_id" INTEGER,
  "zone_id" INTEGER,
  "household_type" INTEGER,
  "walls_material" INTEGER,
  "floor_material" INTEGER,
  "ceiling_material" INTEGER,
  "households_in_land" INTEGER,
  "land_using" INTEGER,
  "household_sq_meters" INTEGER,
  "household_using" INTEGER,
  "family_member_owner" INTEGER,
  "family_member_owner_2" INTEGER,
  "subsidy_or_program" INTEGER,
  "credit" INTEGER,
  "paying_credit" INTEGER,
  "water_source" INTEGER,
  "water_distribution" INTEGER,
  "waste_disposal" INTEGER,
  "electricity" INTEGER,
  "rooms" INTEGER,
  "bathrooms" INTEGER,
  "families_in_household" INTEGER,
  "reason_to_share" INTEGER,
  "cooking_energy_source" INTEGER,
  "heating_energy_source" INTEGER,
  "hot_water_energy_source" INTEGER,
  "less_than_8_blocks_public_transport" INTEGER,
  "less_than_20_blocks_educational_center" INTEGER,
  "less_than_20_blocks_health_center" INTEGER,


### Updates

In [7]:
engine.execute("""
UPDATE environment.fact_household_casen SET family_member_owner = NULL where family_member_owner = -1;
UPDATE environment.fact_household_casen SET family_member_owner_2 = NULL where family_member_owner_2 = -1;
UPDATE environment.fact_household_casen SET subsidy_or_program = NULL where subsidy_or_program = -1;
UPDATE environment.fact_household_casen SET credit = NULL where credit = -1;
UPDATE environment.fact_household_casen SET paying_credit = NULL where paying_credit = -1;
UPDATE environment.fact_household_casen SET reason_to_share = NULL where reason_to_share = -1;
UPDATE environment.fact_household_casen SET zone_id = NULL where zone_id = -1;
""")

In [8]:
engine.execute("""
ALTER TABLE environment.fact_household_casen
  ADD COLUMN date_id INTEGER; 
""")

engine.execute("""
UPDATE environment.fact_household_casen
SET date_id = dim_date.id
FROM public.dim_date
WHERE dim_date.the_year = environment.fact_household_casen.year
      AND dim_date.month_of_year = 1
      AND dim_date.day_of_month = 1
""")


### Dims

The following fields use Binary Survey Response, a shared dimension already created 
* less_than_8_blocks_public_transport
* less_than_20_blocks_educational_center
* less_than_20_blocks_health_center
* less_than_20_blocks_market
* less_than_20_blocks_atm
* less_than_20_blocks_sports_center
* less_than_20_blocks_green_areas
* less_than_20_blocks_community_equipment
* less_than_20_blocks_pharmacy

```xml
   <InlineTable alias="field_name">
      <ColumnDefs>
        <ColumnDef name="id" type="Numeric" />
        <ColumnDef name="description" type="String" />
        <ColumnDef name="es_description" type="String" />
      </ColumnDefs>
      
      <Rows>
        <Row>
          <Value column="id">0</Value>
          <Value column="description">NA</Value>
          <Value column="es_description">NA</Value>
        </Row>  
        <Row>
          <Value column="id">1</Value>
          <Value column="description">NO</Value>
          <Value column="es_description">NO</Value>
        </Row>
        <Row>
          <Value column="id">2</Value>
          <Value column="description">YES</Value>
          <Value column="es_description">SI</Value>
        </Row>
      </Rows>
    </InlineTable>
```

The following fields use Frecuency Survey Response, a shared dimension  
* affected_by_acoustic_contamination
* affected_by_air_contamination
* affected_by_river_or_lake_contamination
* affected_by_public_water_source_contamination
* affected_by_graffitis_or_advertising
* affected_by_accumulation_of_trash_public_areas
* affected_by_animal_or_insect_plague

```xml
   <InlineTable alias="field_name">
      <ColumnDefs>
        <ColumnDef name="id" type="Numeric" />
        <ColumnDef name="description" type="String" />
        <ColumnDef name="es_description" type="String" />
      </ColumnDefs>
      <Rows>
        <Row>
          <Value column="id">1</Value>
          <Value column="description">nunca</Value>
          <Value column="es_description">nunca</Value>
        </Row>
      <Row>
          <Value column="id">2</Value>
          <Value column="description">pocas veces</Value>
          <Value column="es_description">pocas veces</Value>
        </Row>
      <Row>
          <Value column="id">3</Value>
          <Value column="description">muchas veces</Value>
          <Value column="es_description">muchas veces</Value>
        </Row>
      <Row>
          <Value column="id">4</Value>
          <Value column="description">siempre</Value>
          <Value column="es_description">siempre</Value>
        </Row>
      </Rows>
    </InlineTable>
``` 

The following fields use Energy Source Survey Response, a shared dimension  
* cooking_energy_source
* heating_energy_source
* hot_water_energy_source

```xml
   <InlineTable alias="field_name">
      <ColumnDefs>
        <ColumnDef name="id" type="Numeric" />
        <ColumnDef name="description" type="String" />
        <ColumnDef name="es_description" type="String" />
      </ColumnDefs>
      <Rows>
        <Row>
          <Value column="id">1</Value>
          <Value column="description">gas (licuado o de cañería)</Value>
          <Value column="es_description">gas (licuado o de cañería)</Value>
        </Row>
      <Row>
          <Value column="id">2</Value>
          <Value column="description">parafina o petróleo</Value>
          <Value column="es_description">parafina o petróleo</Value>
        </Row>
      <Row>
          <Value column="id">3</Value>
          <Value column="description">leña o derivados (pellets, astillas o briquetas)</Value>
          <Value column="es_description">leña o derivados (pellets, astillas o briquetas)</Value>
        </Row>
      <Row>
          <Value column="id">4</Value>
          <Value column="description">carbón</Value>
          <Value column="es_description">carbón</Value>
        </Row>
      <Row>
          <Value column="id">5</Value>
          <Value column="description">electricidad</Value>
          <Value column="es_description">electricidad</Value>
        </Row>
      <Row>
          <Value column="id">6</Value>
          <Value column="description">energía solar</Value>
          <Value column="es_description">energía solar</Value>
        </Row>
      <Row>
          <Value column="id">7</Value>
          <Value column="description">no usa combustible o fuente de energía</Value>
          <Value column="es_description">no usa combustible o fuente de energía</Value>
        </Row>
      <Row>
          <Value column="id">8</Value>
          <Value column="description">no tiene sistema</Value>
          <Value column="es_description">no tiene sistema</Value>
        </Row>
      </Rows>
    </InlineTable>
``` 

### Other dimensions

In [9]:
dims = list(df1)
dims.remove('comuna_id')
dims.remove('expr')
dims.remove('expc')
dims.remove('year')
dims.remove('comuna_datachile_id')

dims.remove('less_than_8_blocks_public_transport')
dims.remove('less_than_20_blocks_educational_center')
dims.remove('less_than_20_blocks_health_center')
dims.remove('less_than_20_blocks_market')
dims.remove('less_than_20_blocks_atm')
dims.remove('less_than_20_blocks_sports_center')
dims.remove('less_than_20_blocks_green_areas')
dims.remove('less_than_20_blocks_community_equipment')
dims.remove('less_than_20_blocks_pharmacy')

dims.remove('affected_by_acoustic_contamination')
dims.remove('affected_by_air_contamination')
dims.remove('affected_by_river_or_lake_contamination')
dims.remove('affected_by_public_water_source_contamination')
dims.remove('affected_by_graffitis_or_advertising')
dims.remove('affected_by_accumulation_of_trash_public_areas')
dims.remove('affected_by_animal_or_insect_plague')

dims.remove('cooking_energy_source')
dims.remove('heating_energy_source')
dims.remove('hot_water_energy_source')

for d in dims:
    print(d);
    d2 = download_file(remote_path+'ids/',local_path,d+'_id.csv')
    print (inline_dimension_xml(d2, d, 'id', 'value',d))

    


zone_id
Already downloaded. Using: ../data/zone_id_id.csv

<Dimension name="Zone Id" foreignKey="zone_id">
  <Hierarchy hasAll="true">
            
<InlineTable alias="zone_id">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">urbano</Value>
      <Value column="es_description">urbano</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">rural</Value>
      <Value column="es_description">rural</Value>
    </Row>
  </Rows>
</InlineTable>
    
    <Level name="Zone Id" column="id" nameColumn="description" uniqueMembers="true">
      <Annotations>
        <Annotation name="es_caption">Description ES</Annotation>
      </Annotations>
      <Property name="Description ES" column="es_description" />
    </Level>
  </Hierarchy>
</Dimensi